In [78]:
from transformers import DistilBertModel,DistilBertTokenizer
from torchvision import datasets, models, transforms
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import numpy as np
import torch


In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pradyumnjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
max_len=20

In [82]:
sentence = "Musume, haute, couture, K-pop idol girl, full body, beautiful body line, luxurious and beautiful beauty and body, cinematic lighting, detail, Unreal Engine 5, octane render, v-ray render"

In [83]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [84]:
sentence = preprocess_sentence(sentence)

In [96]:
tokens = tokenizer.encode(sentence,add_special_tokens=True,max_length=20,padding = True,truncation=True)
input_ids = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension

In [97]:
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs.last_hidden_state


In [99]:
sentence_embedding = torch.mean(embeddings, dim=1).squeeze(0)

In [102]:
sentence_embedding.size()

torch.Size([768])